In [1]:
#importing required libraries
import os
import glob
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
 
#importing keras libraries
import keras
from keras.models import Model, Sequential
from tensorflow.keras.applications import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.layers import Dense, Activation ,Flatten, Conv2D, MaxPool2D

#importing sklearn libraries
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_val_score

In [2]:
#directory path for full dataset
imagedir ="/home/sanjeev/DL_POC/MlaImg_Data/Malimg_data/"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

Label: 0	Family:       Adialer.C	Number of images: 122
Label: 1	Family:       Agent.FYI	Number of images: 116
Label: 2	Family:       Allaple.A	Number of images: 2949
Label: 3	Family:       Allaple.L	Number of images: 1591
Label: 4	Family:   Alueron.gen!J	Number of images: 198
Label: 5	Family:       Autorun.K	Number of images: 106
Label: 6	Family:     C2LOP.gen!g	Number of images: 200
Label: 7	Family:         C2LOP.P	Number of images: 146
Label: 8	Family:  Dialplatform.B	Number of images: 177
Label: 9	Family:       Dontovo.A	Number of images: 162
Label:10	Family:        Fakerean	Number of images: 381
Label:11	Family:   Instantaccess	Number of images: 431
Label:12	Family:      Lolyda.AA1	Number of images: 213
Label:13	Family:      Lolyda.AA2	Number of images: 184
Label:14	Family:      Lolyda.AA3	Number of images: 123
Label:15	Family:       Lolyda.AT	Number of images: 159
Label:16	Family:     Malex.gen!J	Number of images: 136
Label:17	Family:   Obfuscator.AD	Number of images: 142
Label:18

In [4]:
width, height, channels = (224, 224, 3) #image input shape
X = np.zeros((num_samples, width, height, channels))
cnt = 0
paths_list = []
print("Processing images...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        paths_list.append(os.path.join(os.getcwd(),img_file))
        img = load_img(img_file, target_size=(224, 224))
        x = img_to_array(img) #image to array
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

Processing images...
Images processed: 9339


In [5]:
X.shape, y.shape

((9339, 224, 224, 3), (9339,))

In [6]:
y

array([ 0.,  0.,  0., ..., 24., 24., 24.])

In [7]:
vgg = Sequential()
vgg.add(Conv2D(3, (3, 3), padding='same', input_shape=(224, 224, 3)))
vgg.add(Activation('relu'))

_vgg = VGG19(weights='imagenet', include_top=False)

counter=0
for layer in _vgg.layers:
    layer.trainable = False
    counter+=1

print("VGG's ", counter , " layers are not added to the layer")
vgg.add(_vgg)
print("done")

VGG's  22  layers are not added to the layer
done


In [8]:
vgg.add(Flatten())

vgg.add(Dense(4096,activation='relu'))
vgg.add(Dense(4096,activation='relu'))
vgg.add(Dense(128,activation="relu"))

vgg.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
vgg.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 3)       84        
                                                                 
 activation (Activation)     (None, 224, 224, 3)       0         
                                                                 
 vgg19 (Functional)          (None, None, None, 512)   20024384  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 4096)              102764544 
                                                                 
 dense_1 (Dense)             (None, 4096)              16781312  
                                                                 
 dense_2 (Dense)             (None, 128)               5

In [10]:
%%time
#extracting features
features = vgg.predict(X)

CPU times: user 2h 42min 28s, sys: 47min 28s, total: 3h 29min 57s
Wall time: 4min 33s


In [11]:
features.shape

(9339, 128)

In [12]:
features[0]

array([ 0.        ,  4.5472836 ,  0.        ,  0.        , 14.476263  ,
        0.        ,  5.0486436 ,  0.        ,  7.0406284 ,  0.58900285,
        0.        ,  0.        ,  4.8245587 ,  0.        ,  5.052854  ,
        0.        ,  0.        ,  0.44362295, 12.306916  ,  9.2417    ,
        3.3928814 ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  2.5474813 ,  1.8338861 ,
        4.362805  ,  0.        ,  3.9975915 ,  4.2189946 ,  0.        ,
        5.3378057 ,  0.        ,  0.9316397 ,  0.        ,  2.736334  ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        6.1610427 ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  2.5388927 ,  0.        ,  7.0984216 ,  0.8671255 ,
        5.0843573 ,  9.156063  ,  0.        ,  0.3706305 ,  0.        ,
        3.3284407 ,  0.        ,  5.5515327 ,  0.        , 16.337133  ,
        0.        ,  8.203415  ,  6.7478313 ,  3.9578495 ,  3.71

In [13]:
#np.save('/home/sanjeev/DL_Exp_Kajal/Finetune_stack_features/MalImg/vgg16_feat_malimg_128.npy', features)

In [14]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.naive_bayes import GaussianNB

In [15]:
feat = np.load("/home/sanjeev/DL_Exp_Kajal/Finetune_stack_features/MalImg/vgg16_feat_malimg_128.npy")

In [16]:
y

array([ 0.,  0.,  0., ..., 24., 24., 24.])

In [17]:
#classification model creation using different classifiers
def classify(model, x, y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.10, random_state=31)
    print(X_train.shape, y_train.shape)
    print(X_test.shape, y_test.shape)
    
    start_time = time.time()
    
    model.fit(X_train, y_train)   
    print("Training Accuracy: ", round(model.score(X_train, y_train)*100,3))
    
    end_time = time.time()
    training_time = end_time - start_time 
    print("Training Time:", training_time, "seconds")
    ###################################################################
    start_time = time.time() 
    print("Testing Accuracy: ", round(model.score(X_test, y_test)*100,3))
    end_time = time.time()
    training_time = end_time - start_time 
    print("Testing Time:", training_time, "seconds")
    ####################################################################
    acc = round(model.score(X_test, y_test)*100,3)
    
    score = cross_val_score(model, x, y, cv=5)
    print("Model Accuracy for cross validation:", round(np.mean(score)*100, 2))
    cv = round(np.mean(score)*100, 2)
    
    pred = model.predict(X_test)
    print('Precision:', int(round(precision_recall_fscore_support(y_test, pred, average='micro')[0]*100, 0)))
    print('Recall:', int(round(precision_recall_fscore_support(y_test, pred, average='micro')[1]*100, 0)))
    print('F1_Score:', int(round(precision_recall_fscore_support(y_test, pred, average='micro')[2]*100, 0)))
    ls = precision_recall_fscore_support(y_test, pred, average='micro')
    pre, rec, f1 = int(round(ls[0]*100, 0)), int(round(ls[1]*100, 0)), int(round(ls[2]*100, 0))
    print('*---------------------------*')
    return [acc, pre, rec, f1, cv]

classifiers = [KNeighborsClassifier(),
               SVC(random_state=31),
               RandomForestClassifier(random_state=31),
               MLPClassifier(random_state=31, max_iter=500),
               ExtraTreeClassifier(random_state=31),
               GaussianNB()]

ls_acc, ls_pre, ls_rec, ls_f1, ls_cv = [],[],[],[],[]

for classifier in classifiers:
    print(classifier)
    values = classify(classifier, feat, y)       
    
    ls_acc.append(values[0])
    ls_pre.append(values[1])
    ls_rec.append(values[2])
    ls_f1.append(values[3])
    ls_cv.append(values[4])
    
print([*ls_acc, *ls_pre, *ls_rec, *ls_f1, *ls_cv])

KNeighborsClassifier()
(8405, 128) (8405,)
(934, 128) (934,)
Training Accuracy:  97.466
Training Time: 2.9984567165374756 seconds
Testing Accuracy:  96.36
Testing Time: 0.41150522232055664 seconds
Model Accuracy for cross validation: 96.28
Precision: 96
Recall: 96
F1_Score: 96
*---------------------------*
SVC(random_state=31)
(8405, 128) (8405,)
(934, 128) (934,)
Training Accuracy:  98.56
Training Time: 5.671456336975098 seconds
Testing Accuracy:  97.859
Testing Time: 0.4934852123260498 seconds
Model Accuracy for cross validation: 97.66
Precision: 98
Recall: 98
F1_Score: 98
*---------------------------*
RandomForestClassifier(random_state=31)
(8405, 128) (8405,)
(934, 128) (934,)
Training Accuracy:  100.0
Training Time: 6.063085079193115 seconds
Testing Accuracy:  95.503
Testing Time: 0.04027986526489258 seconds
Model Accuracy for cross validation: 95.09
Precision: 96
Recall: 96
F1_Score: 96
*---------------------------*
MLPClassifier(max_iter=500, random_state=31)
(8405, 128) (8405,)

In [18]:
# #classification model creation using different classifiers
# def classify(model, x, y):
#     X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.10, random_state=31)
#     print(X_train.shape, y_train.shape)
#     print(X_test.shape, y_test.shape)

#     model.fit(X_train, y_train)   
#     print("Training Accuracy: ", round(model.score(X_train, y_train)*100,3))
#     print("Testing Accuracy: ", round(model.score(X_test, y_test)*100,3))
#     acc = round(model.score(X_test, y_test)*100,3)
    
#     score = cross_val_score(model, x, y, cv=5)
#     print("Model Accuracy for cross validation:", round(np.mean(score)*100, 2))
#     cv = round(np.mean(score)*100, 2)
    
#     pred = model.predict(X_test)
#     print('Precision:', int(round(precision_recall_fscore_support(y_test, pred, average='micro')[0]*100, 0)))
#     print('Recall:', int(round(precision_recall_fscore_support(y_test, pred, average='micro')[1]*100, 0)))
#     print('F1_Score:', int(round(precision_recall_fscore_support(y_test, pred, average='micro')[2]*100, 0)))
#     ls = precision_recall_fscore_support(y_test, pred, average='micro')
#     pre, rec, f1 = int(round(ls[0]*100, 0)), int(round(ls[1]*100, 0)), int(round(ls[2]*100, 0))
#     print('*---------------------------*')
#     return [acc, pre, rec, f1, cv]

# classifiers = [KNeighborsClassifier(),
#                SVC(random_state=31),
#                RandomForestClassifier(random_state=31),
#                MLPClassifier(random_state=31, max_iter=500),
#                ExtraTreeClassifier(random_state=31),
#                GaussianNB()]

# ls_acc, ls_pre, ls_rec, ls_f1, ls_cv = [],[],[],[],[]

# for classifier in classifiers:
#     print(classifier)
#     values = classify(classifier, feat, y)       
    
#     ls_acc.append(values[0])
#     ls_pre.append(values[1])
#     ls_rec.append(values[2])
#     ls_f1.append(values[3])
#     ls_cv.append(values[4])
    
# print([*ls_acc, *ls_pre, *ls_rec, *ls_f1, *ls_cv])

KNeighborsClassifier()
(8405, 128) (8405,)
(934, 128) (934,)
Training Accuracy:  97.466
Testing Accuracy:  96.36
Model Accuracy for cross validation: 96.28
Precision: 96
Recall: 96
F1_Score: 96
*---------------------------*
SVC(random_state=31)
(8405, 128) (8405,)
(934, 128) (934,)
Training Accuracy:  98.56
Testing Accuracy:  97.859
Model Accuracy for cross validation: 97.66
Precision: 98
Recall: 98
F1_Score: 98
*---------------------------*
RandomForestClassifier(random_state=31)
(8405, 128) (8405,)
(934, 128) (934,)
Training Accuracy:  100.0
Testing Accuracy:  95.503
Model Accuracy for cross validation: 95.09
Precision: 96
Recall: 96
F1_Score: 96
*---------------------------*
MLPClassifier(max_iter=500, random_state=31)
(8405, 128) (8405,)
(934, 128) (934,)
Training Accuracy:  100.0
Testing Accuracy:  97.43
Model Accuracy for cross validation: 97.24
Precision: 97
Recall: 97
F1_Score: 97
*---------------------------*
ExtraTreeClassifier(random_state=31)
(8405, 128) (8405,)
(934, 128) 